In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.utils import shuffle

In [ ]:
network = pd.read_table("breast_correlation_v2.txt", sep='\t', header=None, names=['src', 'dest', 'r', 'p'])
network.head()

In [ ]:
def print_stat(network):
    src_nodes = set(np.unique(network.src.values))
    dest_nodes = set(np.unique(network.dest.values))
    print "number of all nodes: " + str(len(src_nodes | dest_nodes))
    print "number of src nodes: " + str(np.unique(network.src.values).shape[0])
    print "number of dest nodes: " + str(np.unique(network.dest.values).shape[0])
    print "number of src_nodes - dest_nodes: " + str(len(src_nodes - dest_nodes))
    print "number of dest_nodes - src_nodes: " + str(len(dest_nodes - src_nodes))
    print "number of all edges: " + str(network.shape[0])


In [ ]:
# draw histogram of edge weights (correlation)
plt.figure()
network.r.plot.hist(alpha=0.5, bins=20)
plt.axvline(x=0.5, color='red')
plt.axvline(x=-0.5, color='red')
plt.show()

plt.figure()
network.p.plot.hist(alpha=0.5, bins=20)
plt.show()

# subnetwork

cut off network by edge weight (correlation)

In [ ]:
network_cutoff5 = network[abs(network.r) > 0.5]
print_stat(network_cutoff5)

In [ ]:
# construct graph
def construct_graph(df):
    G = nx.DiGraph()
    for idx, row in df.iterrows():
        G.add_edge(row.src, row.dest, corr=row.r, pval=row.p)

    # print graph properties
    print nx.number_weakly_connected_components(G)
    print nx.is_directed_acyclic_graph(G)
    print nx.isolates(G)
    print nx.is_strongly_connected(G)
    print nx.number_strongly_connected_components(G)
    print nx.is_semiconnected(G)
    
    return G

In [ ]:
plt.figure()
plt.bar(range(len(nx.degree_histogram(G))), nx.degree_histogram(G))
plt.show()

In [ ]:
network_cutoff5.to_csv("breast_cancer_correlation_cutoff_0_6", sep='\t')

# dataframe creation

In [ ]:
deep_input = pd.read_table('deep_input_c3_pert_v2LogN', sep='\t', header=0)
#deep_input = deep_input.loc[:, ['Gene:Cell', 'T/F']]
#deep_input_tmp = deep_input['Gene:Cell'].str.split(':', expand=True)
#deep_input = pd.concat([deep_input_tmp, deep_input['T/F']], axis=1)
#deep_input
deep_input.columns = ['gene', 'cell', 'label']
deep_input.head()

In [ ]:
truefalse = np.zeros(network.shape[0], dtype=np.int32)
truefalse.shape

In [ ]:
def mark_clusters():
    network = pd.read_table("../breast_cancer_corr_essential_marked", sep='\t', header=0, index_col=0)
    #network['c'] = np.zeros(network.shape[0], dtype=np.int32)

    for c in range(1, 4):
        df_c = pd.read_table('../clusters/mini_c{0}.txt'.format(c), sep='\t', header=None, names=['gene'])
        df_c = df_c['gene'].str.split(':', expand=True)
        df_c.columns = ['gene', 'cell']
        print "df_c" + str(c) 
        #rint df_c
        
        diff_set = set(df_c.gene.values) - set(network.src.values)
        
        if len(diff_set) != 0:
            for gene in diff_set:
                row = pd.Series(np.zeros((network.shape[1], ), dtype=object), index=network.columns)
                row['src'] = gene
                row['dest'] = None
                network = network.append(row, ignore_index=True)
            ### END - for
        ### END - if
        
        print "difference " + str(len(diff_set))
        
        c_col = np.zeros(network.shape[0], dtype=np.int32)
        for idx, row in network.iterrows():
            if row.src in df_c.gene.values:
                c_col[idx] = 1
        network['c{0}'.format(c)] = c_col
        ### END - for
    ### END - for
    
    network['f'] = np.zeros(network.shape[0], dtype=np.int32)
    
    for f in range(1, 6):
        df_f = pd.read_table('../clusters/mini_f{0}.txt'.format(f), sep='\t', header=None, names=['gene', 'cluster'])
        diff_set = set(df_f.gene.values) - set(network.src.values)
        
        if len(diff_set) != 0:
            for gene in diff_set:
                row = pd.Series(np.zeros((network.shape[1], ), dtype=object), index=network.columns)
                row['src'] = gene
                row['dest'] = None
                network = network.append(row, ignore_index=True)
        
        print "difference " + str(len(diff_set))
        
        for idx, row in network.iterrows():
            if row.src in df_f.gene.values:
                network.at[idx, 'f'] = f
    ### END - for
    
    for c in range(1, 4):
        for f in range(1, 6):
            idx = network[(network['c{0}'.format(c)] == 1) | (network['f'] == f)].index
            col = np.zeros(network.shape[0], dtype=np.int32)
            col[idx] = 1
            network['c{0}f{1}'.format(c, f)] = col
    return network

In [ ]:
network = mark_clusters()
network.to_csv('../breast_cancer_cluster_marked_cf', sep='\t')

In [ ]:
def make_gene_df():
    db = pd.read_table('../deep_input_c3_pert_v2LogN', header=0, sep='\t')
    db = db[['Gene:Cell', 'T/F']]
    db[['gene', 'cell']] = db['Gene:Cell'].str.split(':', expand=True)
    db.drop('Gene:Cell', axis=1, inplace=True)
    db = db[['gene', 'cell', 'T/F']]
    db = db.sort_values('gene', axis=0).reset_index()
    print db.head()
    db.drop(['level_0', 'index'], axis=1, inplace=True)
    print db.head()
    
    db_2 = db.drop_duplicates(subset='gene')
    db_2.head()
    genes = db_2['gene'].values
    gene_df = db_2
    gene_df.head()
    #gene_df.set_index('gene', inplace=True)
    gene_df.loc[:, 'total'] = 0
    gene_df.loc[:, 'ess'] = 0
    gene_df.loc[:, 'cells'] = ''

    for gene in genes:
        cnt = db[db['gene'] == gene]['T/F'].sum()
        gene_df.loc[gene, 'ess'] = cnt
        gene_df.loc[gene, 'total'] = db[db['gene'] == gene].shape[0]
    gene_df = gene_df[['total', 'ess']]
    gene_df.to_csv('gene_essential_cnt', sep='\t')
    return gene_df

In [ ]:
node_df = pd.read_table("../breast_cancer_node_df", header=0, index_col=0)
node_df = node_df[['src_ess', 'in_central', 'out_central', 'close', 'between']]

gene_df.reset_index(inplace=True)
node_df.reset_index(inplace=True)
print gene_df.head()
print node_df.head()

ess_pool = gene_df.gene.values
node_df.loc[:, 'mark'] = 0
mark_idx = []
for idx, row in node_df.iterrows():
    if row.src in ess_pool:
        mark_idx.append(idx)

node_df.loc[mark_idx, 'mark'] = 1
node_df

experiment_pool_centrality = node_df[node_df.mark == 1].drop('mark', axis=1)
experiment_pool_centrality.to_csv("../experiment_pool_centrality", sep='\t')

experiment_pool_centrality.plot.box()

In [1]:
node_df = pd.read_table('../breast_cancer_all_node_df', header=0, index_col=0)

NameError: name 'pd' is not defined

In [33]:
# make essentiality dataframe

node_df = pd.read_table('../all_node_centrality.tsv', sep='\t', header=0, index_col=0)
for c in ['c1', 'c3', 'c3_cs']:
    node_df.loc[:, c + 'ess'] = None
    for ess in ['Essen', 'Non']:
        with open('../Input_{0}_{1}.txt'.format(ess, c), 'r') as f:
            ls = f.read().split('\n')

            for gene in ls:
                if gene in node_df.index:
                    if ess == 'Essen':
                        node_df.loc[gene, c + '_ess'] = 1
                    else:
                        node_df.loc[gene, c + '_ess'] = 0
            ### END - for gene
        ### END - with open
    ### END - for ess
### END - for c



node_df.loc[:, 'normal_pred_ess'] = 0
node_df.loc[:, 'tumor_pred_ess'] = 0
for ess in ['N', 'T']:
    with open('../Gene_c3_top_case_3x_relu_1_{0}_0.5_3.tsv'.format(ess), 'r') as f:
        ls = f.read().split('\n')
        for gene in ls:
            if gene in node_df.index:
                if ess == 'T':
                    node_df.loc[gene, 'tumor_pred_ess'] = 1
                else:
                    node_df.loc[gene, 'normal_pred_ess'] = 1
            ### END - for gene
        ### END - with open
    ### END - for ess
### END - for c
                    
node_df.apply(pd.to_numeric, errors='coerce', downcast='integer')

node_df = node_df.drop(['in_central', 'out_central', 'close', 'between'], axis=1)
node_df.to_csv('../all_node_essentiality.tsv', sep='\t')

,in_central,out_central,close,between,c1_ess,c3_ess,c3_cs_ess
src,,,,,,,
A1BG,0.000000,0.000090,0.000120,0.000000e+00,None,None,None
A2M,0.000271,0.000090,0.000348,3.657683e-06,1,None,None
A2ML1,0.000090,0.000090,0.000090,4.073143e-08,0,None,None
AAAS,0.000181,0.000090,0.000250,4.236069e-07,0,0,None
AAMP,0.000090,0.000451,0.000615,3.665829e-08,None,1,None
AARS2,0.000181,0.000181,0.000181,5.967155e-07,None,0,None
AARSD1,0.000090,0.000181,0.000181,1.629257e-08,0,0,None
AASDH,0.000000,0.000090,0.000120,0.000000e+00,0,None,None
AASDHPPT,0.000181,0.000090,0.001416,6.837450e-06,None,None,None


In [8]:
ess_df = pd.read_table('../all_node_essentiality.tsv', sep='\t', header=0, index_col=0)
list(ess_df[ess_df['c1_ess'] == 0].index)
ess_df.loc['FOLR3']

c1_ess             NaN
c3_ess             NaN
c3_cs_ess          NaN
normal_pred_ess    0.0
tumor_pred_ess     0.0
Name: FOLR3, dtype: float64

In [5]:
a = [1, 2, 3]
list(a)

[1, 2, 3]

# pariwise shortest path among cluster nodes

In [15]:
def construct_graph(set_cluster=True):
	""" Constructs networkx graph from network DataFrame. 
	If set_cluster is True, cluster information is added to the nodes.

	args:
		set_cluster (bool) : whether to put cluster information to nodes

	returns:
		networkx constructed from edge information data frame

	"""

	# network_df should also have clustered information
	df = pd.read_table('../breast_cancer_cluster_marked_cf', sep='\t', header=0, index_col=0)
	ess_dic, c1_dic, c2_dic, c3_dic, f_dic = {}, {}, {}, {}, {}
	G = nx.DiGraph()
	# add edges to the graph
	for idx, row in df.iterrows():
		G.add_edge(row.src, row.dest, r=row.r, p=row.p)
		if set_cluster:
			ess_dic[row.src] = row.src_ess
			c1_dic[row.src] = row.c1
			c2_dic[row.src] = row.c2
			c3_dic[row.src] = row.c3
			f_dic[row.src] = row.f
	### END - for
	if set_cluster:
		nx.set_node_attributes(G, 'ess', ess_dic)
		nx.set_node_attributes(G, 'c1', c1_dic)
		nx.set_node_attributes(G, 'c2', c2_dic)
		nx.set_node_attributes(G, 'c3', c3_dic)
		nx.set_node_attributes(G, 'f', f_dic)

	try:
		G.remove_node(np.nan)
	except NetworkXError:
		print "node requested to remove does not exists"

	# remove node that are not in the main conneted_component
	to_be_removed = isolated_nodes(G)
	for n in to_be_removed:
		G.remove_node(n)

	return G


def isolated_nodes(G):
	""" Returns the nodes that are not in the largest weakly connected component.

	args:
		G (nx.DiGraph) : graph

	returns:
		set of nodes that are not in the largest weakly connected component

	"""
	to_be_removed = set()
	idx = 0
	for comp in nx.weakly_connected_components(G):
		if idx != 0:
			to_be_removed.union(comp)
		idx += 1
	return to_be_removed

In [16]:
def pairwise_shortest_path(G, node_list):
	""" Calculate pairwise shortest path length for given nodes (node_list) in graph G. 
	If a path exists in either way, use that path length. 
	If no path exists, use np.inf as length.

	args:
		G (NetworkX graph) : directed or undirected graph
		node_list (list) : list of nodes to calculate pairwise shortest length

	returns:
		list of shortest path lengts in unrolled fasion

	"""
	list_len = len(node_list)
	path_len_list = []
	for i in range(list_len):
		for k in range(i+1, list_len):
			n1 = node_list[i]
			n2 = node_list[k]
			try:
				length = nx.shortest_path_length(G, source=n1, target=n2, weight=None)
			except nx.NetworkXNoPath:
				try:
					length = nx.shortest_path_length(G, source=n2, target=n1, weight=None)
				except nx.NetworkXNoPath:
					length = np.inf
			path_len_list.append(length)
		### END - for k
	### END - for i
	return path_len_list
### pairwise_shortest_path



In [17]:
def compare_distance():
	G = construct_graph()
	df = pd.read_table('../breast_cancer_all_node_df', sep='\t', header=0, index_col=None)
	
	all_nodes = df.src.values
	len_df = pd.DataFrame()
	for c in range(1, 4):
		for f in range(1, 6):
			cluster = df[df['c{0}f{1}'.format(c, f)] == 1]['src'].values
			random_nodes = list(shuffle(all_nodes, n_samples=cluster.shape[0]))
			
			len_df['c{0}f{1}'.format(c, f)] = pd.Series(pairwise_shortest_path(G, list(cluster)))
			len_df['rand_c{0}f{1}'.format(c, f)] = pd.Series(pairwise_shortest_path(G, random_nodes))
		### END - for f
	### END - for c
	return len_df

In [38]:
len_df = compare_distance()

In [39]:
for c in range(1, 4):
    for f in range(1, 6):
        clust = 'rand_c{0}f{1}'.format(c, f)
        print clust, float(len_df[(len_df[clust] != np.inf) & (len_df[clust].notnull())][clust].shape[0]) / len_df[len_df[clust].notnull()][clust].shape[0]

rand_c1f1 0.0122772277228
rand_c1f2 0.0073145245559
rand_c1f3 0.00435643564356
rand_c1f4 0.00327255726975
rand_c1f5 0.0108910891089
rand_c2f1 0.0
rand_c2f2 0.0
rand_c2f3 0.0121457489879
rand_c2f4 0.030303030303
rand_c2f5 0.00215053763441
rand_c3f1 0.00792079207921
rand_c3f2 0.00888192267503
rand_c3f3 0.00732673267327
rand_c3f4 0.00915122397621
rand_c3f5 0.0253465346535


In [42]:
for c in range(1, 4):
    for f in range(1, 6):
        clust = 'c{0}f{1}'.format(c, f)
        print clust, float(len_df[(len_df[clust] != np.inf) & (len_df[clust].notnull())][clust].shape[0]) / len_df[len_df[clust].notnull()][clust].shape[0]

c1f1 0.0552475247525
c1f2 0.066091954023
c1f3 0.0772277227723
c1f4 0.0841514726508
c1f5 0.0681188118812
c2f1 0.0473684210526
c2f2 0.619047619048
c2f3 0.0337381916329
c2f4 0.363636363636
c2f5 0.0774193548387
c3f1 0.050099009901
c3f2 0.0626959247649
c3f3 0.0669306930693
c3f4 0.0828185769847
c3f5 0.0643564356436


In [ ]:
for c in range(1, 4):
    for f in range(1, 6):
        cluster = df[df['c{0}f{1}'.format(c, f)] == 1]['src'].values
        print sorted(set(cluster))
        print sorted(set(list(shuffle(all_nodes, n_samples=cluster.shape[0]))))
        break
    break
        #print cluster

In [ ]:
for n in range(10):
    len_list = pairwise_shortest_path(G, list(shuffle(all_nodes, n_samples=20, random_state=n)))
    len_arr = np.array(len_list)
    print len_arr[len_arr != np.inf].shape[0] / float(len_arr.shape[0])

c = 2
print '***'
for f in range(1, 6):
    cluster = df[df['c{0}f{1}'.format(c, f)] == 1]['src'].values
    len_list = pairwise_shortest_path(G, list(cluster))
    print len_list
    len_arr = np.array(len_list)
    print float(len_arr[len_arr != np.inf].shape[0]) / len_arr.shape[0]

In [4]:
def path_exist_ratio(len_list):
	len_arr = np.array(len_list)
	return len_arr[len_arr != np.inf].shape[0] / float(len(len_list))


In [ ]:
for c in range(1, 4):
    for f in range(1, 6):
        for n in range(1000):
            len_list = pairwise_shortest_path(G, list(shuffle(all_nodes, n_samples=, random_state=n)))
            len_arr = np.array(len_list)
            print len_arr[len_arr != np.inf].shape[0] / float(len_arr.shape[0])

In [29]:
# make easy for use in R dataframe (convert format)
path_df = pd.read_table('../undirected_path_length_essentiality.tsv', 
                        sep='\t', header=0, index_col=0)
new_df = pd.DataFrame(columns=['group', 'distance'])
for col in path_df.columns:

    temp_df = pd.DataFrame({'group' : col, 
                            'distance' : path_df[col][path_df[col].notnull()]
                           })
    new_df = pd.concat([new_df, temp_df], axis=0, ignore_index=True)
new_df = new_df[['group', 'distance']]
new_df.to_csv('../undirected_path_length_essentiality_for_r.tsv', sep='\t')


In [27]:
print new_df[new_df['group'] == 'c3_ess'].shape
print new_df[new_df['group'] == 'c3_non'].shape

(5841, 2)
(20236, 2)
